In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-152742", 
                   subscription_id="48a74bb7-9950-4cc1-9caa-5d50f995cc55",
                   resource_group="aml-quickstarts-152742")

exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-152742
Azure region: southcentralus
Subscription id: 48a74bb7-9950-4cc1-9caa-5d50f995cc55
Resource group: aml-quickstarts-152742


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

vm_size = "Standard_D2_V2"
max_nodes = 4
cpu_cluster_name = "p1-cluster"

compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size, max_nodes=max_nodes)
cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [38]:
from azureml.core import Environment

sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

In [69]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C": uniform(0.0, 5.0),
    "--max_iter": choice(100, 200, 300, 400)    
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
#est = ### YOUR CODE HERE ###
rc = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=cpu_cluster,
                      #arguments=[],
                      environment=sklearn_env
                     )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=rc, 
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=4
)

In [70]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hdr = exp.submit(config=hyperdrive_config)

In [71]:
RunDetails(hdr).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [75]:
import joblib
# Get your best run and save the model from that run.

best_run = hdr.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Regularization Strength:', best_run_metrics['Regularization Strength:'])
print('\n Max iterations:', best_run_metrics['Max iterations:'])

#best_run.download_file()

Best Run Id:  HD_004b70d8-7f94-4807-9cfd-2d4602e51e42_12

 Accuracy: 0.9139354212187424

 Regularization Strength: 2.856729890159631

 Max iterations: 300


In [1]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

tabular_data = TabularDatasetFactory.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [6]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(tabular_data)

In [10]:
tabular_data.to_pandas_dataframe().head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


In [12]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=tabular_data,
    label_column_name="y",
    n_cross_validations=5,
    compute_target=cpu_cluster,
    max_concurrent_iteration=4,
    iteration_timeout_minutes=5,
    iterations=25
)

In [13]:
# Submit your automl run

exp = Experiment(ws, "autml_project1")
run = exp.submit(config=automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on p1-cluster with default configuration
Running on remote compute: p1-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
autml_project1,AutoML_80203911-68f5-497b-abbf-1156e06772b5,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+-------------------------

In [20]:
# Retrieve and save your best automl model.

best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
#print('\n Regularization Strength:', best_run_metrics['Regularization Strength:'])
#print('\n Max iterations:', best_run_metrics['Max iterations:'])

AttributeError: 'AutoMLRun' object has no attribute 'get_best_run_by_primary_metric'

In [21]:
dir(run)

['DELIM',
 'EXPERIMENT_ID_PATH',
 'EXPERIMENT_NAME_PATH',
 'RUN_PATH',
 'TID_FMT',
 'WORKSPACE_FMT',
 '_DEFAULT_GET_CONTENT_TIMEOUT',
 '_RUNSOURCE_PROPERTY',
 '_WAIT_COMPLETION_POLLING_INTERVAL_MAX',
 '_WAIT_COMPLETION_POLLING_INTERVAL_MIN',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cached_best_model',
 '_cached_best_run',
 '_cleanup',
 '_client',
 '_container',
 '_context_manager',
 '_create',
 '_download_artifact_contents_to_string',
 '_dto_to_run',
 '_experiment',
 '_experiment_url',
 '_fetch_guardrails',
 '_formatted_tid',
 '_from_run_dto',
 '_get_all_automl_child_runs',
 '_get_aml_token_auth',
 '_get_automl_child_iteration',
 '_g

In [23]:
run.complete()

In [25]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
autml_project1,AutoML_80203911-68f5-497b-abbf-1156e06772b5,automl,Running,Link to Azure Machine Learning studio,Link to Documentation
